# Uber Driver Data Analysis

This notebook analyzes driver performance, earnings, and behavior patterns from the Uber hackathon dataset.

## 1. Import Libraries and Load Data

In [ ]:
!pip install -U pip
!pip install pandas matplotlib seaborn openpyxl

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100

In [ ]:
# ---------------------------
# Load all sheets from the Excel file
# ---------------------------
file_path = "uber_hackathon_v2_mock_data.xlsx"
sheets = pd.read_excel(file_path, sheet_name=None)

# Assign sheets to variables
earners = sheets['earners']
rides_trips = sheets['rides_trips']
earnings_daily = sheets['earnings_daily']
incentives_weekly = sheets['incentives_weekly']
cancellation_rates = sheets['cancellation_rates']
surge_by_hour = sheets.get('surge_by_hour')        # optional
weather_daily = sheets.get('weather_daily')        # optional
heatmap_sheet = sheets.get('heatmap')              # optional
jobs_like = sheets.get('jobs_like')                # optional

# Convert datetime columns
rides_trips['start_time'] = pd.to_datetime(rides_trips['start_time'])
rides_trips['end_time'] = pd.to_datetime(rides_trips['end_time'])
earnings_daily['date'] = pd.to_datetime(earnings_daily['date'])

print(f"Loaded {len(sheets)} sheets from {file_path}")
print(f"\nEarners: {len(earners)} records")
print(f"Rides/Trips: {len(rides_trips)} records")
print(f"Daily Earnings: {len(earnings_daily)} records")
print(f"Weekly Incentives: {len(incentives_weekly)} records")

## 2. Earnings vs Working Time

In [ ]:
# ---------------------------
# 1. Earnings vs Working Time
# ---------------------------
plt.figure(figsize=(8,6))
sns.scatterplot(x='rides_duration_mins', y='total_net_earnings', data=earnings_daily, alpha=0.5)
plt.title('Earnings vs. Working Time')
plt.xlabel('Rides Duration (mins)')
plt.ylabel('Total Net Earnings (€)')
plt.tight_layout()
plt.show()

## 3. Trips per Day Distribution

In [ ]:
# ---------------------------
# 2. Trips per Day Distribution
# ---------------------------
plt.figure(figsize=(8,6))
sns.histplot(earnings_daily['trips_count'], bins=15, kde=False)
plt.title('Trips per Day Distribution')
plt.xlabel('Trips per Day')
plt.ylabel('Number of Drivers')
plt.tight_layout()
plt.show()

## 4. Hourly Earnings Curve

In [ ]:
# ---------------------------
# 3. Hourly Earnings Curve
# ---------------------------
rides_trips['hour'] = rides_trips['start_time'].dt.hour
hourly_earnings = rides_trips.groupby('hour')['net_earnings'].mean().reset_index()
plt.figure(figsize=(8,6))
sns.lineplot(x='hour', y='net_earnings', data=hourly_earnings, marker='o')
plt.title('Hourly Earnings Curve')
plt.xlabel('Hour of Day')
plt.ylabel('Average Net Earnings (€)')
plt.tight_layout()
plt.show()

## 5. Idle Time Analysis

In [ ]:
# ---------------------------
# 4. Idle Time Analysis
# ---------------------------
rides_trips_sorted = rides_trips.sort_values(['driver_id','start_time'])
rides_trips_sorted['next_start'] = rides_trips_sorted.groupby('driver_id')['start_time'].shift(-1)
rides_trips_sorted['idle_time_mins'] = (rides_trips_sorted['next_start'] - rides_trips_sorted['end_time']).dt.total_seconds()/60

# Filter out negative and extreme values
rides_trips_filtered = rides_trips_sorted[(rides_trips_sorted['idle_time_mins'] > 0) & 
                                           (rides_trips_sorted['idle_time_mins'] < 300)]

plt.figure(figsize=(8,6))
sns.scatterplot(x='idle_time_mins', y='net_earnings', data=rides_trips_filtered, alpha=0.5)
plt.title('Idle Time vs Earnings')
plt.xlabel('Idle Time (mins)')
plt.ylabel('Net Earnings (€)')
plt.tight_layout()
plt.show()

## 6. Weekly Incentives Effect

In [ ]:
# ---------------------------
# 5. Weekly Incentives Effect
# ---------------------------
weekly_earnings = earnings_daily.groupby('earner_id').agg({'total_net_earnings':'sum'}).reset_index()
weekly_data = weekly_earnings.merge(incentives_weekly, on='earner_id', how='left')
plt.figure(figsize=(8,6))
sns.scatterplot(x='bonus_eur', y='total_net_earnings', data=weekly_data, alpha=0.5)
plt.title('Weekly Incentives vs Total Earnings')
plt.xlabel('Bonus (€)')
plt.ylabel('Total Net Earnings (€)')
plt.tight_layout()
plt.show()

## 7. Driver Rating vs Workload

In [ ]:
# ---------------------------
# 6. Driver Rating vs Workload
# ---------------------------
driver_workload = rides_trips.groupby('driver_id')['duration_mins'].mean().reset_index()
rating_workload = driver_workload.merge(earners[['earner_id','rating']], 
                                         left_on='driver_id', 
                                         right_on='earner_id', 
                                         how='left')
plt.figure(figsize=(8,6))
sns.scatterplot(x='duration_mins', y='rating', data=rating_workload, alpha=0.5)
plt.title('Driver Rating vs Average Workload')
plt.xlabel('Average Trip Duration (mins)')
plt.ylabel('Driver Rating')
plt.tight_layout()
plt.show()

## 8. Cancellation Rate vs Surge

In [ ]:
# ---------------------------
# 7. Cancellation Rate vs Surge
# ---------------------------
if surge_by_hour is not None:
    # Add hour column to cancellation_rates if needed
    # For this analysis, we'll aggregate by city
    cancellation_city = cancellation_rates.groupby('city_id').agg({
        'cancellation_rate_pct': 'mean'
    }).reset_index()
    
    surge_city = surge_by_hour.groupby('city_id').agg({
        'surge_multiplier': 'mean'
    }).reset_index()
    
    cancellation_surge = cancellation_city.merge(surge_city, on='city_id', how='left')
    
    plt.figure(figsize=(8,6))
    sns.scatterplot(x='surge_multiplier', y='cancellation_rate_pct', 
                    data=cancellation_surge, s=100)
    plt.title('Cancellation Rate vs Surge Multiplier (by City)')
    plt.xlabel('Average Surge Multiplier')
    plt.ylabel('Average Cancellation Rate (%)')
    plt.tight_layout()
    plt.show()
else:
    print("Surge data not available")

## 9. Fatigue Heatmap - Trips by Hour and Day

In [ ]:
# ---------------------------
# 8. Fatigue Heatmap
# ---------------------------
rides_trips['day_of_week'] = rides_trips['start_time'].dt.day_name()
heatmap_data = rides_trips.pivot_table(index='hour', 
                                        columns='day_of_week', 
                                        values='ride_id', 
                                        aggfunc='count')

# Reorder columns to show days in proper order
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
heatmap_data = heatmap_data[[col for col in day_order if col in heatmap_data.columns]]

plt.figure(figsize=(12,8))
sns.heatmap(heatmap_data, cmap='YlOrRd', annot=True, fmt='g', cbar_kws={'label': 'Number of Trips'})
plt.title('Fatigue Heatmap: Trips per Hour by Day of Week')
plt.xlabel('Day of Week')
plt.ylabel('Hour of Day')
plt.tight_layout()
plt.show()

## 10. Driver Performance Table - Trips, Duration & Profits per Day

In [ ]:
# ---------------------------
# Driver Performance Table: Trips, Duration & Profits per Day
# ---------------------------

# Create a comprehensive table showing driver performance metrics per day
driver_daily_summary = earnings_daily.copy()

# Add driver information (rating, etc.)
driver_daily_summary = driver_daily_summary.merge(
    earners[['earner_id', 'rating']],
    on='earner_id',
    how='left'
)

# Add day of week for better readability
driver_daily_summary['day_of_week'] = driver_daily_summary['date'].dt.day_name()
driver_daily_summary['date_formatted'] = driver_daily_summary['date'].dt.strftime('%Y-%m-%d')

# Create the summary table with key metrics
performance_table = driver_daily_summary[[
    'earner_id', 'date_formatted', 'day_of_week', 'trips_count',
    'rides_duration_mins', 'total_net_earnings', 'rating'
]].copy()

# Sort by driver ID and date
performance_table = performance_table.sort_values(['earner_id', 'date_formatted'])

# Rename columns for better readability
performance_table.columns = [
    'Driver ID', 'Date', 'Day of Week', 'Daily Trips',
    'Total Duration (mins)', 'Total Earnings (€)', 'Driver Rating'
]

print("Driver Performance Summary - Trips, Duration & Profits per Day")
print("=" * 80)
print(f"Total Records: {len(performance_table)}")
print(f"Unique Drivers: {performance_table['Driver ID'].nunique()}")
print(f"Date Range: {performance_table['Date'].min()} to {performance_table['Date'].max()}")
print("\nFirst 20 records:")
print("-" * 80)

# Display the table with proper formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

display(performance_table.head(20))

In [ ]:
# ---------------------------
# Additional Statistics and Insights
# ---------------------------

print("\n" + "="*80)
print("DRIVER PERFORMANCE STATISTICS")
print("="*80)

# Overall statistics
print("\n📊 OVERALL PERFORMANCE METRICS:")
print("-" * 40)
print(f"Average Daily Trips per Driver: {performance_table['Daily Trips'].mean():.2f}")
print(f"Average Daily Duration per Driver: {performance_table['Total Duration (mins)'].mean():.2f} minutes ({performance_table['Total Duration (mins)'].mean()/60:.1f} hours)")
print(f"Average Daily Earnings per Driver: €{performance_table['Total Earnings (€)'].mean():.2f}")
print(f"Average Driver Rating: {performance_table['Driver Rating'].mean():.2f}")

# Top performers
print(f"\n🏆 TOP PERFORMING DRIVERS (by total earnings):")
print("-" * 40)
top_earners = performance_table.groupby('Driver ID').agg({
    'Daily Trips': 'sum',
    'Total Duration (mins)': 'sum',
    'Total Earnings (€)': 'sum',
    'Driver Rating': 'first'
}).sort_values('Total Earnings (€)', ascending=False).head(5)

top_earners.columns = ['Total Trips', 'Total Duration (mins)', 'Total Earnings (€)', 'Rating']
display(top_earners)

# Efficiency metrics (earnings per trip, earnings per hour)
print(f"\n💡 EFFICIENCY METRICS:")
print("-" * 40)
efficiency_table = performance_table.copy()

# Calculate efficiency metrics properly (handling zero cases)
efficiency_table['Earnings per Trip (€)'] = efficiency_table.apply(
    lambda row: row['Total Earnings (€)'] / row['Daily Trips'] if row['Daily Trips'] > 0 else 0, axis=1
)
efficiency_table['Earnings per Hour (€)'] = efficiency_table.apply(
    lambda row: row['Total Earnings (€)'] / (row['Total Duration (mins)'] / 60) if row['Total Duration (mins)'] > 0 else 0, axis=1
)

# Calculate averages excluding zeros
valid_trips = efficiency_table[efficiency_table['Daily Trips'] > 0]
valid_duration = efficiency_table[efficiency_table['Total Duration (mins)'] > 0]

print(f"Average Earnings per Trip: €{valid_trips['Earnings per Trip (€)'].mean():.2f}")
print(f"Average Earnings per Hour: €{valid_duration['Earnings per Hour (€)'].mean():.2f}")

# Day of week analysis
print(f"\n📅 PERFORMANCE BY DAY OF WEEK:")
print("-" * 40)
day_performance = performance_table.groupby('Day of Week').agg({
    'Daily Trips': 'mean',
    'Total Duration (mins)': 'mean',
    'Total Earnings (€)': 'mean'
}).round(2)

day_performance.columns = ['Avg Daily Trips', 'Avg Duration (mins)', 'Avg Earnings (€)']
display(day_performance)

In [ ]:
# ---------------------------
# Export Detailed Performance Table
# ---------------------------

# Create a more detailed table for export/analysis
detailed_performance_table = performance_table.copy()

# Add efficiency metrics
detailed_performance_table['Earnings per Trip (€)'] = detailed_performance_table.apply(
    lambda row: round(row['Total Earnings (€)'] / row['Daily Trips'], 2) if row['Daily Trips'] > 0 else 0, axis=1
)

detailed_performance_table['Earnings per Hour (€)'] = detailed_performance_table.apply(
    lambda row: round(row['Total Earnings (€)'] / (row['Total Duration (mins)'] / 60), 2) if row['Total Duration (mins)'] > 0 else 0, axis=1
)

print("\n" + "="*80)
print("DETAILED DRIVER PERFORMANCE TABLE")
print("="*80)
print(f"Total records in table: {len(detailed_performance_table)}")
print(f"Columns: {list(detailed_performance_table.columns)}")

print(f"\nSample of detailed table (first 10 records):")
print("-" * 80)
display(detailed_performance_table.head(10))

# Optionally save to Excel
try:
    detailed_performance_table.to_excel('driver_performance_table.xlsx', index=False)
    print(f"\n✅ Table exported to 'driver_performance_table.xlsx'")
except Exception as e:
    print(f"\n⚠️ Could not export to Excel: {e}")

# Summary statistics
active_drivers = detailed_performance_table[detailed_performance_table['Daily Trips'] > 0]
print(f"\n📋 FINAL SUMMARY:")
print("-" * 30)
print(f"• Total records: {len(detailed_performance_table)}")
print(f"• Active driver-days: {len(active_drivers)}")
print(f"• Unique drivers: {detailed_performance_table['Driver ID'].nunique()}")
print(f"• Date range: {detailed_performance_table['Date'].min()} to {detailed_performance_table['Date'].max()}")
print(f"• Total trips across all drivers: {detailed_performance_table['Daily Trips'].sum()}")
print(f"• Total earnings across all drivers: €{detailed_performance_table['Total Earnings (€)'].sum():.2f}")
print(f"• Total working time: {detailed_performance_table['Total Duration (mins)'].sum():.0f} minutes ({detailed_performance_table['Total Duration (mins)'].sum()/60:.1f} hours)")

In [ ]:
# ---------------------------
# Load DataFrame from CSV file
# ---------------------------

# Method 1: Basic CSV loading
# Replace 'your_file.csv' with your actual CSV file path
csv_file_path = "data\data-1759588094558.csv"

try:
    # Load CSV into DataFrame
    df = pd.read_csv(csv_file_path)

    print(f"Successfully loaded CSV: {csv_file_path}")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\nFirst 5 rows:")
    display(df.head())

except FileNotFoundError:
    print(f"File '{csv_file_path}' not found. Please check the file path.")
except Exception as e:
    print(f"Error loading CSV: {e}")



In [ ]:
df.sort_values(['city', 'latitude', 'longitude'], ascending=[False, False, False])
df[df['city']==1].describe()
df.plot(kind='scatter', x='longitude', y='latitude', alpha=0.5,)
# Locationaldf=df.set_index(['city', 'latitude', 'longitude'])["predicted_eph"].to_frame()
# Locationaldf.head()

In [ ]:
cityArray= []
for i in range(1, 6):
    cityArray.append(df[df['city']==i])
cityArray[0].describe()


In [ ]:
cityArray[3].plot(kind='scatter', x='longitude', y='latitude', alpha=0.1)
rides_trips[rides_trips['city_id']==4].plot(kind='scatter', x='drop_lon', y='drop_lat', alpha=0.1)
rides_trips[rides_trips['city_id']==4].plot(kind='scatter', x='pickup_lon', y='pickup_lat', alpha=0.1)


In [ ]:
i=1
latdif = cityArray[i]['latitude'].max() - cityArray[i]['latitude'].min()
longdif = cityArray[i]['longitude'].max() - cityArray[i]['longitude'].min()
cityArray[i]["latsector"] = 0
cityArray[i]["longsector"] = 0
for j in range(1, 101):
    lat = cityArray[i].loc[cityArray[i]['latitude'].between(cityArray[i]['latitude'].quantile(0.01*(i-1)), cityArray[i]['latitude'].quantile(0.01*i)), 'latsector'] = j
    # lat = cityArray[i]['latitude'].min() + (latdif * j / 100)
    if cityArray[i]['latitude'].between(lat - (latdif / 100), lat).any():
        cityArray[i].loc[cityArray[i]['latitude'].between(lat - (latdif / 100), lat), 'latsector'] = j
    # long = cityArray[i]['longitude'].min() + (longdif * j / 100)
    long = cityArray[i].loc[cityArray[i]['longitude'].between(cityArray[i]['longitude'].quantile(0.01*(i-1)), cityArray[i]['longitude'].quantile(0.01*i)), 'longsector'] = j
    if cityArray[i]['longitude'].between(long - (longdif / 100), long).any():
        cityArray[i].loc[cityArray[i]['longitude'].between(long - (longdif / 100), long), 'longsector'] = j
cityArray[i].sort_values(['latsector', 'longsector'], ascending=[True, True], inplace=True)
cityArray[i].describe()


In [ ]:
test = cityArray[i].groupby(['latsector', 'longsector'])['predicted_eph'].mean().unstack()
sns.heatmap(test, cmap='YlOrRd', annot=False, cbar_kws={'label': 'Average predicted_eph'})